In [13]:
# -*- coding: utf-8 -*-

## standard packages
import numpy as np
import os
import datetime

## Config Parster for initiation
import configparser

## HTTP scraping packages
from lxml import html
import requests

#import httplib2
import urllib

from bs4 import BeautifulSoup, SoupStrainer
import json

## Database
import sqlite3

from importlib import reload


In [14]:
import src.utils as utils

In [9]:
pagesConfig = configparser.ConfigParser()
pagesConfig.read("config/siteSpecifics.ini")

for section in pagesConfig.sections():
    print(section)
    #for option in pagesConfig.options(section):
        #print(option, pagesConfig.get(section, option))
    #    print (section, option)

WebPages
additionalAdInfo
dbFilepath
dbInit


In [10]:
searchPage = pagesConfig.get('WebPages', 'mobile.de')
print(searchPage)

http://suchen.mobile.de/fahrzeuge/search.html?scopeId=C&damageUnrepaired=NO_DAMAGE_UNREPAIRED&scopeId=C&isSearchRequest=true&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING&pageNumber=


In [52]:
reload(utils)
database.close()
database = utils.DataBase('src/dataBase/carStats.db')

scraping = utils.htmlScraping(searchPage, database)

In [54]:
scraping.setPriceStepList(0, 300, 300)
scraping.getPriceStepList()

[0, 300]

In [55]:
dataIDs = scraping.scrapeAdIDs()
print(dataIDs.shape)

(971, 4)


In [56]:
scraping.scrapeWithID()

Error during translating/writing Information to DB at: 381
['247880469'
 'https://suchen.mobile.de/fahrzeuge/details.html?id=247880469&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&maxPrice=300&minPrice=0&pageNumber=21&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING'
 ' href9hrefOpel Sintra "GLS"hrefInserat online seit 24.07.2017 16:19href299 €hrefEZ 03/1999, 339.000\xa0km, 85\xa0kW\xa0(116\xa0PS)hrefVan / Minibus, Diesel, Schaltgetriebe, 4/5 Türenhref≈\u20097,9\u2009l/100km (komb.), ≈\u2009209\u2009g CO₂/km (komb.)hrefAutomarkt Pershiahref(href3href)hrefDE-09120 Chemnitz, HändlerhrefFinanzierunghref, hrefVersicherunghrefGeparkthrefParkenhrefKontakt'
 '24.07.2017']
near "GLS": syntax error
Error during translating/writing Information to DB at: 435
['245252846'
 'https://suchen.mobile.de/fahrzeuge/details.html?id=245252846&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&maxPrice=300&minPrice=0&pageNumber=24&scopeId=C&sortOption.sortBy=cre

In [57]:
someString = ' href11hrefFord Focus Trend "Euro 3"hrefInserat online seit 23.07.2017 12:28href299 €hrefEZ 04/1999, 180.500\xa0km, 85\xa0kW\xa0(116\xa0PS)hrefLimousine, Benzin, Schaltgetriebe, HU 04/2017, 2/3 Türenhref≈\u20097,6\u2009l/100km (komb.), ≈\u2009181\u2009g CO₂/km (komb.)hrefZimmermannhref(href41href)hrefDE-88046 Friedrichshafen, HändlerhrefFinanzierunghref, hrefVersicherunghrefGeparkthrefParkenhrefKontakt'

In [58]:
someString.replace('"', '**')

' href11hrefFord Focus Trend **Euro 3**hrefInserat online seit 23.07.2017 12:28href299 €hrefEZ 04/1999, 180.500\xa0km, 85\xa0kW\xa0(116\xa0PS)hrefLimousine, Benzin, Schaltgetriebe, HU 04/2017, 2/3 Türenhref≈\u20097,6\u2009l/100km (komb.), ≈\u2009181\u2009g CO₂/km (komb.)hrefZimmermannhref(href41href)hrefDE-88046 Friedrichshafen, HändlerhrefFinanzierunghref, hrefVersicherunghrefGeparkthrefParkenhrefKontakt'

In [39]:
#payload = {'scopeId': ['C', 'C'],
#           'damageUnrepaired': 'NO_DAMAGE_UNREPAIRED',
#           'isSearchRequest': 'true',
#           'sortOption.sortBy': 'creationTime',
#           'sortOption.sortOrder': 'DESCENDING'}

def getAdIdList(searchPage, priceStepList, DB):
    payload=dict()
    dataIDs = np.zeros([0, 4])
    dbDataIDs = np.array(DB.execute("""SELECT adID from car""")) # ToDo: make a variable out of the name of the DB

    for step in range(len(priceStepList)-1):
        payload['minPrice'] = priceStepList[step]
        payload['maxPrice'] = priceStepList[step+1]

        for i in range(50): # 50 is the maximum possible number of pages
            payload['pageNumber'] = i+1
            req = requests.get(searchPage, params=payload) # create searchPageURL

            soup = BeautifulSoup(req.text, "lxml")
            for link in soup.find_all('a'):
                if link.has_attr('data-ad-id') and int(link['data-ad-id']) not in dbDataIDs:

                    if dataIDs.shape[0] == 0 or link['data-ad-id'] not in dataIDs[:, 0]:
                        onPos = link.get_text('href').find('online seit ')
                        onlineSince = link.get_text('href')[onPos+12 : onPos+22]
                        dataIDs = np.vstack([dataIDs, [link['data-ad-id'], link['href'], link.get_text('href'), onlineSince]])
        print( step, dataIDs.shape )
        
    return dataIDs
        
        
dataIDs = getAdIdList(searchPage, [0, 100, 200], database)
print(len(dataIDs))

0 (107, 4)
1 (763, 4)
763


In [23]:
dbDataIDs = np.array(database.execute("""SELECT adID from car""")) # ToDo: make a variable out of the name of the DB
dbDataIDs

array([[130918008],
       [198987495],
       [225012466],
       ..., 
       [248088793],
       [248088819],
       [248088850]])

In [36]:
'248269787' in np.ones([0, 4])#dataIDs[:, 0]

C:\Users\Florian\Anaconda2\envs\ipykernel_py3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


False

In [31]:
dataIDs[:, 0]

array(['248269787', '248267306', '248264575', '248252813', '246252021',
       '244467187', '248174697', '248171857', '248148941', '248144493',
       '248109072', '248096330', '248043051', '248037885', '248023448',
       '248023132', '247639279', '248001578', '247998463', '247956139',
       '246383095', '247875251', '247874602', '247874018', '247843047',
       '247836580', '247828802', '247795217', '247697745', '247604053',
       '247592974', '247590114', '247589077', '247564920', '247525545',
       '247524576', '247522826', '247431201', '247413157', '246551441',
       '247393462', '247392745', '247355769', '247351657', '247322635',
       '247320569', '247314188', '247301288', '247293613', '247251769',
       '247236342', '247213399', '246890093', '246860053', '246806549',
       '246795228', '246764470', '246741763', '246740939', '246703292',
       '246612414', '246566321', '246496416', '246384941', '246367408',
       '246354921', '246243883', '246188097', '246149836', '2461

In [35]:
allInfos = utils.getInfoFromPage(dataIDs[0, 1])

In [36]:
allInfos['description']

'ich hatte ihn während meines Bauvorhabens, am Caddy funktioniert alles, aber der Rost hat ihn stark zugesetzt. deshalb an Bastler. Rest TÜV bis Oktober 2017. \r\nübrigens: http://www.bussgeld-info.de/bussgeldkatalog-tuev-papiere/\r\n2-4 Monate ohne TÜV 15€; 4-8 Monate 25€, ab 8 gibts Punkte ;-)\r\nAch und nur nebenbei für die Sparfüchse: oben steht Verbrauch 5,9l/100km, der fährt bei mir mit etwa 5,2 Liter\r\nDer Caddy ist angemeldet und kann mit meiner Zulassung gegen 100€ Kaution überführt werden.\r\nIch bin zurzeit in Elternzeit und daher fast immer Daheim. einfach anrufen'

In [37]:
allInfos['firstSeen'] = dataIDs[0][3]
allInfos['lastSeen'] = str(datetime.datetime.now().date)
allInfos

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '399',
  'specifics': {'make': 'Volkswagen', 'model': 'Caddy'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '7',
 'adFirstRegYear': '2000',
 'adId': 248088793,
 'adImage': 'https://i.ebayimg.com/00/s/OTAwWDE2MDA=/z/0N8AAOSwrk5Zey9u/$_23.JPG',
 'adImageBase64': 'aHR0cHM6Ly9pLmViYXlpbWcuY29tLzAwL3MvT1RBd1dERTJNREE9L3ovME44QUFPU3dyazVaZXk5dS8k',
 'adPrice': '399',
 'adSegment': 'Car',
 'adSpecificsFuel': 'DIESEL',
 'adSpecificsMake': 'Volkswagen',
 'adSpecificsMakeId': 25200,
 'adSpecificsMakeModel': 'Volkswagen Caddy',
 'adSpecificsMakeModelBase64': 'Vm9sa3N3YWdlbiBDYWRkeQ==',
 'adSpecificsModel': 'Caddy',
 'adSpecificsModelDescription': 'Caddy SDI 9K LKW',
 'adSpecificsModelId': 9,
 'airbag': 'No Information',
 'category': 'Van / Minibus',
 'climatisation': 'No Information',
 'countryVersion': 'No Information',
 'cubicCapacity': '1.896\xa0cm³',
 'dealerCity': 'Westhausen',
 'dealerCountry': 'DE',
 'deale

In [60]:
def getSourceCode(page):

    # try to open web page
    try:
        req = requests.get(page)
        response = req.text
    except Exception as e:
        return "Loading the web page has failed: ", e

    # initialize soup
    soup = BeautifulSoup(response, 'lxml')
    siteInfo = soup.decode()
    
    return soup, siteInfo

soup, sourceCode = getSourceCode(searchPage+'2')

In [61]:
position = sourceCode.find('rbt-page-forward')
print(sourceCode[position-100:position+1000])

n></li><li class="pref-next u-valign-sub"><span class="btn btn--orange btn--s next-resultitems-page rbt-page-forward" data-href="https://suchen.mobile.de/fahrzeuge/search.html?damageUnrepaired=NO_DAMAGE_UNREPAIRED&amp;isSearchRequest=true&amp;pageNumber=3&amp;scopeId=C&amp;sortOption.sortBy=creationTime&amp;sortOption.sortOrder=DESCENDING" data-target="default" data-touch="link" title="Zur nächsten Seite">Weitere Treffer<i class="gicon-next-white-s icon--s"></i></span></li></ul></div></div>
<div class="adv D_88_LOWER_SBANNER_de" id="D_88_LOWER_SBANNER">
<div class="ad_container" id="D_88_LOWER_SBANNER-container" style="width:auto">
<script type="text/javascript">
            googletag.cmd.push(function() {
                googletag.display('D_88_LOWER_SBANNER-container');
            });
        </script>
</div>
</div>
<div class="cBox cBox--content" id="motorTalkPromoBox"><div class="cBox-header"><h2 class="h3">MOTOR-TALK</h2><hr/></div><div class="cBox-body"><div class="g-row"><div c

In [78]:
#soup.find(attr={'class':"btn btn--orange btn--s next-resultitems-page rbt-page-forward"})
for link in soup.find_all('span'):
    #if link.has_attr("rbt-page-forward"):
    print (link.attrs)

{'class': ['claim']}
{}
{'class': ['header-nav-link', 'header-nav-link-toggle', 'mde-react-header__dropdown-toggle'], 'data-se': 'header-nav-link-toggle__hdsrch', 'tabindex': '0', 'data-js-header-toggle': 'true'}
{'class': ['font-gibson-semibold']}
{'class': ['header-nav-link', 'header-nav-link-toggle', 'mde-react-header__dropdown-toggle'], 'data-se': 'header-nav-link-toggle__hdoff', 'tabindex': '0', 'data-js-header-toggle': 'true'}
{'class': ['font-gibson-semibold']}
{'class': ['header-nav-link', 'header-nav-link-toggle', 'mde-react-header__dropdown-toggle'], 'data-se': 'header-nav-link-toggle__hdinf', 'tabindex': '0', 'data-js-header-toggle': 'true'}
{'class': ['font-gibson-semibold']}
{'class': ['header-meta-action', 'header-meta-action-toggle', 'mde-react-header__dropdown-toggle'], 'data-js-header-toggle': 'true', 'tabindex': '0', 'data-se': 'header-meta__dealer-menu'}
{}
{'class': ['header-meta-action', 'header-meta-action-toggle', 'mde-react-header__dropdown-toggle'], 'data-js-he

In [74]:
link.attrs

{'class': ['link--hidden'], 'data-se': 'footer-small__DesktopVersion'}

## initiate Database

In [ ]:
#connection = sqlite3.connect('src/dataBase/carStats.db')
#cursor = connection.cursor()

# delete 
#cursor.execute("""DROP TABLE car;""")

#cursor.execute(sql_create_command)

# never forget this, if you want the changes to be saved:
#connection.commit()

#connection.close()

In [69]:
%pwd

'H:\\GitHub_Projects\\UsedCarStats'

In [90]:
from importlib import reload
reload(utils)

<module 'src.utils' from 'H:\\GitHub_Projects\\UsedCarStats\\src\\utils.py'>

In [29]:
#database.close()
database = utils.DataBase('src/dataBase/carStats.db')

In [30]:
database.connect()

In [31]:
database.getTableNames()

[('car',)]

In [32]:
adIDList = database.execute("""SELECT adID FROM car""")
231941829 in np.array(adIDList)

True

In [19]:
cursor = database._cursor

In [22]:
cursor.execute("""SELECT adID FROM car""")
#cursor.fetchall()

In [28]:
database.close()

In [95]:
database.deleteAllContents()

In [96]:
cursor = database.getCursor()
cursor.description

In [97]:
cursor.execute("""DROP TABLE car;""")


In [98]:
attributes = dict()
for option in pagesConfig.options('dbInit'):
    attributes[option] = pagesConfig.get('dbInit', option)
database.createTable(attributes)
cursor = database.getCursor()
cursor.description

In [99]:
database.getTableNames()

[('car',)]

In [100]:
database.execute("""SELECT * FROM car""")

[]

In [101]:
dataIDs.shape

(4874, 4)

In [102]:
def insertIntoDB(dictionary, superKey=''):
    columns = ''
    values = ''
    for key, val in dictionary.items():
        if type(val) == dict:
            c, v = insertIntoDB(val, superKey=superKey + key)
            columns += c
            values += v
        elif key.lower() in attributes.keys():
            columns += superKey + key + ', '
            values += '"' + str(val) + '"' + ', '

    
    return columns, values

for i in range(dataIDs.shape[0]):
    try:
        allInfos = utils.getInfoFromPage(dataIDs[i, 1])
        allInfos['firstSeen'] = dataIDs[i][3]
        allInfos['lastSeen'] = str(datetime.datetime.now().date)

        columns, values = insertIntoDB(allInfos)

        database.execute('''INSERT INTO car ( %s ) VALUES ( %s );''' %(columns[:-2], values[:-2]) )
    except Exception as e:
        print (i)
        print (e)

database.execute("""SELECT * FROM car""")

20
Expecting value: line 1 column 2 (char 1)
62
near "Klimaautomatik": syntax error
97
Expecting value: line 1 column 2 (char 1)
105
near "7": syntax error
124
Expecting value: line 1 column 2 (char 1)
130
Expecting value: line 1 column 2 (char 1)
239
Expecting value: line 1 column 2 (char 1)
270
Expecting value: line 1 column 2 (char 1)
303
Expecting value: line 1 column 2 (char 1)
346
Expecting value: line 1 column 2 (char 1)
365
Expecting value: line 1 column 2 (char 1)
453
Expecting value: line 1 column 2 (char 1)
456
Expecting value: line 1 column 2 (char 1)
462
near "90657": syntax error
533
Expecting value: line 1 column 2 (char 1)
535
Expecting value: line 1 column 2 (char 1)
538
Expecting value: line 1 column 2 (char 1)
548
Expecting value: line 1 column 2 (char 1)
647
near "A": syntax error
649
Expecting value: line 1 column 2 (char 1)
769
near "Klima": syntax error
801
Expecting value: line 1 column 2 (char 1)
846
Expecting value: line 1 column 2 (char 1)
888
Expecting value

KeyboardInterrupt: 

In [104]:
result = database.execute("""SELECT * FROM car""")
len(result)

1421

In [105]:
database.save()
database.close()

In [88]:
i = 42
allInfos = utils.getInfoFromPage(dataIDs[i, 1])
allInfos['firstSeen'] = dataIDs[i][3]
allInfos['lastSeen'] = str(datetime.datetime.now().date)
allInfos

{'ad': {'condition': 'USED',
  'currency': 'EUR',
  'price': '990',
  'specifics': {'make': 'Opel', 'model': 'Astra'}},
 'adCondition': 'USED',
 'adCurrency': 'EUR',
 'adFirstRegMonth': '3',
 'adFirstRegYear': '1996',
 'adId': 248086929,
 'adImage': 'https://i.ebayimg.com/00/s/MTIwMFgxNjAw/z/s0QAAOSwv89ZezGz/$_23.JPG',
 'adImageBase64': 'aHR0cHM6Ly9pLmViYXlpbWcuY29tLzAwL3MvTVRJd01GZ3hOakF3L3ovczBRQUFPU3d2ODlaZXpHei8k',
 'adPrice': '990',
 'adSegment': 'Car',
 'adSpecificsFuel': 'PETROL',
 'adSpecificsHsn': '35',
 'adSpecificsMake': 'Opel',
 'adSpecificsMakeId': 19000,
 'adSpecificsMakeModel': 'Opel Astra',
 'adSpecificsMakeModelBase64': 'T3BlbCBBc3RyYQ==',
 'adSpecificsModel': 'Astra',
 'adSpecificsModelDescription': 'Astra Caravan 1,6 GL SSD, TÜV 07/2019',
 'adSpecificsModelId': 5,
 'adSpecificsTsn': '321',
 'airbag': 'Front-Airbags',
 'category': 'Kombi',
 'climatisation': 'No Information',
 'countryVersion': 'No Information',
 'cubicCapacity': '1.598\xa0cm³',
 'dealerCity': 'Asperg/

In [86]:
print(dataIDs[i, 1])

https://suchen.mobile.de/fahrzeuge/details.html?id=248088646&damageUnrepaired=NO_DAMAGE_UNREPAIRED&isSearchRequest=true&maxPrice=1000&minPrice=0&pageNumber=1&scopeId=C&sortOption.sortBy=creationTime&sortOption.sortOrder=DESCENDING


In [78]:
cursor = database.getCursor()
cursor.description

(('adid', None, None, None, None, None, None),
 ('adcondition', None, None, None, None, None, None),
 ('adcurrency', None, None, None, None, None, None),
 ('adfirstregmonth', None, None, None, None, None, None),
 ('adfirstregyear', None, None, None, None, None, None),
 ('adprice', None, None, None, None, None, None),
 ('adsegment', None, None, None, None, None, None),
 ('adspecificsfuel', None, None, None, None, None, None),
 ('adspecificshsn', None, None, None, None, None, None),
 ('adspecificsmake', None, None, None, None, None, None),
 ('adspecificsmakeid', None, None, None, None, None, None),
 ('adspecificsmakemodel', None, None, None, None, None, None),
 ('adspecificsmodel', None, None, None, None, None, None),
 ('adspecificsmodeldescription', None, None, None, None, None, None),
 ('adspecificsmodelid', None, None, None, None, None, None),
 ('dealercity', None, None, None, None, None, None),
 ('dealercountry', None, None, None, None, None, None),
 ('dealerzip', None, None, None, N

In [226]:
for i in cursor.description:
    print (i[0])

adid
adcondition
adcurrency
adfirstregmonth
adfirstregyear
adprice
adsegment
adspecificsfuel
adspecificshsn
adspecificsmake
adspecificsmakeid
adspecificsmakemodel
adspecificsmodel
adspecificsmodeldescription
adspecificsmodelid
dealercity
dealercountry
dealerzip
featurevariants
isdealer
sellertype
usersipaddress
category
countryversion
mileage
cubiccapacity
power
fuel
fuelconsumption
emission
numseats
doorcount
transmission
emissionssticker
firstregistration
numberofpreviousowners
hu
climatisation
parkassist
airbag
manufacturercolorname
color
interior
features
description
firstseen
lastseen


In [108]:
connection.close()